In [2]:
import pymc3 as pm
import numpy as np
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from arviz.utils import Numba
Numba.disable_numba()
Numba.numba_flag

False

In [3]:
# 조합 구조를 가진 솔루션의 경우 : 컨텍스트 기반 슬롯머신 문제
# - 각 선택지가 어떤 속성의 조합, 즉 특성량feature에 따라 나타나는 경우

# 예를 들어 2가지 그림 / 2가지 문구 조합이 있다고 했을 때
# xa = (0, 0) xb = (0,1) xc = (1,0), xd = (1,1) 로 표현할 수 있음.
# 또한 각 디자인안 x는 각각 고유의 클릭률 theta_x를 가지고 있다고 가정. 즉, 어떤 디자인안을 사용자에게 표시하면 그 클릭률을 파라미터로 가진 베르누이 분포로부터 클릭, 다시 말해 보상r이 생성됨.

In [4]:
#MCMC를 이용한 로지스틱 회귀 톰슨 샘플링

# - 톰슨 샘플링은 보상 기댓값의 사후 분포만 알면 해당 분포로부터 샘플을 생성하는 것만으로토 간단하게 알고리즘을 구성할 수 있음.
# - 여기서는 톰슨 샘플링에 로지스틱 회귀 모델을 도입
# - 보상기댓값의 사후분포를 로지스틱 회귀 모델을 이용해 추론하는 점이 기존 슬롯머신 문제와 다름.

In [ ]:
arms = [[0, 0], [0, 1], [1, 0], [1, 1]]

class MCMC_GLMTSAgent(object):
    def __init__(self):
        self.counts = [0 for _ in arms]   # 선택가능한 솔루션의 집합
        self.wins = [0 for _ in arms]
        self.phis = np.array([[arm[0], arm[1], 1] for arm in arms]).T   # arms의 각 멤버에 정수 1을 추가
    
    def get_arm(self):
        if 0 in self.counts: return self.counts.index(0)  # 한번도 시도한 적이 없는 솔루션의 경우 그 솔루션 출력
        with pm.Model() as model:   # 각 솔루션의 보상 기대값 theta를 추론 + 사후 분포로부터 샘플이 최대가 되는 솔루션 출력
            w = pm.Normal('w', mu=0, sigma=10, shape=3)
            linpred = pm.math.dot(w, self.phis)
            theta = pm.Deterministic(
                    'theta', 1 / (1 + pm.math.exp(-linpred))
            )
            obs = pm.Binomial(
                'obs', n=self.counts, p=theta, observed=self.wins
            )
            trace = pm.sample(2000, chains=1)
            sample = pm.sample_posterior_predictive( trace, samples=1, model=model, vars=[theta])
        return np.argmax(sample['theta'])
    
    def sample(self, arm_index, reward):
        self.counts[arm_index] += 1
        self.sins[arm_index] += reward
        
        

class Env(object):
    def p(arm):
        x = arm[0] * 0.2 + arm[1]*0.8 -4
        p = 1/(1 + np.exp(-x))
        return p
    
    def react(arm):
        return 1 if np.random.random() < Env.p(arm) else 0
    
    def opt():
        return np.argmax([Env.p(arm) for arm in arms])